# Secure Data Access To S3 Bucket via IAM Policy

# TODO:

1) creating a new bucket called ${bucket}-iam-user with NO priviledges

2) call `aws s3 ls ${bucket}-iam-user` and show that it doesn't work

3) applying a bucket policy that allows access FOR THIS USER

4) call `aws s3 ls ${bucket}-iam-user` and show that it works

In [16]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name='s3', region_name=region)
iam = boto3.Session().client(service_name='iam', region_name=region)

In [2]:
import time
timestamp = int(time.time())

In [3]:
bucket_secure_name = 'bucket-secure-iam-policy-role-{}'.format(timestamp)
print(bucket_secure_name)

bucket-secure-iam-policy-role-1608439293


In [4]:
!aws s3 mb s3://$bucket_secure_name

make_bucket: bucket-secure-iam-policy-role-1608439293


In [5]:
!aws s3 ls s3://$bucket_secure_name

# TODO: split to get name

In [6]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::231218423789:role/TeamRole


In [8]:
policy_name='DSOAWS_SecureBucket_Policy_{}'.format(timestamp)
print(policy_name)

DSOAWS_SecureBucket_Policy_1608439293


In [7]:
# Create the IAM policy
iam_policy_deny = {
        'Version': '2012-10-17',
        'Statement': [{
            'Sid': '',
            'Effect': 'Deny',
            'Action': [
                's3:ListBucket'
            ],
            'Resource': [
                'arn:aws:s3:::{}'.format(bucket_secure_name)
            ]
        }]
    }

In [11]:
import json 

response = iam.put_role_policy(
    RoleName='TeamRole',
    PolicyName=policy_name,
    PolicyDocument=json.dumps(iam_policy_deny)
)

print(response)

time.sleep(5)

{'ResponseMetadata': {'RequestId': '02f1488b-7c9b-4a78-a24e-c63819fb1f22', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '02f1488b-7c9b-4a78-a24e-c63819fb1f22', 'content-type': 'text/xml', 'content-length': '206', 'date': 'Sun, 20 Dec 2020 04:49:43 GMT'}, 'RetryAttempts': 0}}


In [13]:
!aws s3 ls s3://$bucket_secure_name


An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied


In [20]:
response = iam.delete_role_policy(
    RoleName='TeamRole',
    PolicyName=policy_name
)
print(response)

{'ResponseMetadata': {'RequestId': '62eb4cb3-0de1-4676-813e-806f9a68ad41', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '62eb4cb3-0de1-4676-813e-806f9a68ad41', 'content-type': 'text/xml', 'content-length': '212', 'date': 'Sun, 20 Dec 2020 05:05:15 GMT'}, 'RetryAttempts': 0}}


In [21]:
!aws s3 ls s3://$bucket_secure_name